In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

tf.keras.backend.clear_session()


In [2]:
IMG_DIR = 'cleaned_data/ISIC_2019_Training_Input_cleaned'
IMG_SIZE = (224, 224)
data = pd.read_csv("cleaned_data\\combined_GT_MD.csv")
class_cols = ['MEL', 'NV', 'BCC', 'BKL', 'other']


In [3]:
data['label'] = data[class_cols].idxmax(axis=1)
label_map = {name: idx for idx, name in enumerate(class_cols)}
data['label'] = data['label'].map(label_map)

In [4]:
# Extract metadata columns
meta_cols = [c for c in data.columns if c not in ['image', 'label', 'split', 'lesion_id'] + class_cols]


In [5]:
def load_images(image_ids, folder):
    images = []
    for image_id in image_ids:
        path = os.path.join(folder, image_id + '.jpg')
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: image {path} not found or can't be read.")
            continue
        img = cv2.resize(img, IMG_SIZE)
        img = img.astype(np.float32) / 255.0
        images.append(img)
    return np.array(images)


In [6]:
# Split the data into training and validation sets
train_df = data[data['split'] == 'train']
val_df = data[data['split'] == 'val']




In [8]:
X_train_img = load_images(train_df['image'], IMG_DIR)
X_val_img = load_images(val_df['image'], IMG_DIR)

KeyboardInterrupt: 

In [ ]:
X_train_meta = train_df[meta_cols].values
X_val_meta = val_df[meta_cols].values
X_train_meta = X_train_meta.astype('float32')
X_val_meta = X_val_meta.astype('float32')


In [ ]:
y_train = to_categorical(train_df['label'], num_classes=5)
y_val = to_categorical(val_df['label'], num_classes=5)


In [ ]:
print(f"Train images shape: {X_train_img.shape}")
print(f"Train metadata shape: {X_train_meta.shape}")
print(f"Train labels shape: {y_train.shape}")

print(f"Val images shape: {X_val_img.shape}")
print(f"Val metadata shape: {X_val_meta.shape}")
print(f"Val labels shape: {y_val.shape}")


Train images shape: (16284, 224, 224, 3)
Train metadata shape: (16284, 13)
Train labels shape: (16284, 5)
Val images shape: (3588, 224, 224, 3)
Val metadata shape: (3588, 13)
Val labels shape: (3588, 5)


In [ ]:
img_input = Input(shape=(224, 224, 3), name='img_input')
meta_input = Input(shape=(13,), name='meta_input')

In [ ]:
# Load the EfficientNetB0 model without the top layer
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=img_input)
# Freeze the base model layers
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
# Dropout layer for regularization
x = Dropout(0.3)(x)
# Concatenate image features with metadata
x = Concatenate()([x, meta_input])


In [ ]:
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.regularizers import l2


x = BatchNormalization()(x)
# First dense layer with L2 regularization
x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)              
x = Dropout(0.4)(x)                     

# Second dense layer with L2 regularization
x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Output layer with softmax activation
output = Dense(5, activation='softmax')(x)


In [ ]:
# Create the model
model = Model(inputs=[img_input, meta_input], outputs=output)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

checkpoint_path = 'cnn_before.keras'

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)
]
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
callbacks.append(reduce_lr)


In [23]:
# Train the model
history = model.fit(
    {'img_input': X_train_img, 'meta_input': X_train_meta},
    y_train,
    validation_data=(
        {'img_input': X_val_img, 'meta_input': X_val_meta},
        y_val
    ),
    epochs=20,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)



Epoch 1/20
509/509 ━━━━━━━━━━━━━━━━━━━━ 1173s 2s/step - accuracy: 0.2901 - loss: 2.3188 - val_accuracy: 0.4013 - val_loss: 1.8257 - learning_rate: 1.0000e-05
Epoch 2/20
509/509 ━━━━━━━━━━━━━━━━━━━━ 1191s 2s/step - accuracy: 0.3492 - loss: 2.1114 - val_accuracy: 0.3788 - val_loss: 1.8198 - learning_rate: 1.0000e-05
Epoch 3/20
509/509 ━━━━━━━━━━━━━━━━━━━━ 1181s 2s/step - accuracy: 0.4036 - loss: 1.9688 - val_accuracy: 0.4699 - val_loss: 1.6707 - learning_rate: 1.0000e-05
Epoch 4/20
509/509 ━━━━━━━━━━━━━━━━━━━━ 1173s 2s/step - accuracy: 0.4289 - loss: 1.8767 - val_accuracy: 0.4967 - val_loss: 1.6070 - learning_rate: 1.0000e-05
Epoch 5/20
509/509 ━━━━━━━━━━━━━━━━━━━━ 1165s 2s/step - accuracy: 0.4457 - loss: 1.8319 - val_accuracy: 0.4189 - val_loss: 1.7332 - learning_rate: 1.0000e-05
Epoch 6/20
509/509 ━━━━━━━━━━━━━━━━━━━━ 1185s 2s/step - accuracy: 0.4758 - loss: 1.7346 - val_accuracy: 0.5220 - val_loss: 1.5701 - learning_rate: 1.0000e-05
Epoch 7/20
509/509 ━━━━━━━━━━━━━━━━━━━━ 1175s 2s/ste

In [24]:
# Unfreeze the base model for fine-tuning
base_model.trainable = True

In [25]:
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training the model with fine-tuning
history_fine = model.fit(
    {'img_input': X_train_img, 'meta_input': X_train_meta}, 
    y_train,                                                
    validation_data=(
        {'img_input': X_val_img, 'meta_input': X_val_meta},
        y_val
    ),
    epochs=50,
    callbacks=callbacks,
    initial_epoch=20)

Epoch 21/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1375s 3s/step - accuracy: 0.6188 - loss: 1.3292 - val_accuracy: 0.6062 - val_loss: 1.3845 - learning_rate: 1.0000e-05
Epoch 22/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1332s 3s/step - accuracy: 0.6238 - loss: 1.3349 - val_accuracy: 0.5513 - val_loss: 1.5215 - learning_rate: 1.0000e-05
Epoch 23/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1295s 3s/step - accuracy: 0.6400 - loss: 1.2830 - val_accuracy: 0.5619 - val_loss: 1.4953 - learning_rate: 1.0000e-05
Epoch 24/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1294s 3s/step - accuracy: 0.6491 - loss: 1.2770 - val_accuracy: 0.6407 - val_loss: 1.2622 - learning_rate: 1.0000e-05
Epoch 25/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1280s 3s/step - accuracy: 0.6462 - loss: 1.2693 - val_accuracy: 0.6321 - val_loss: 1.3010 - learning_rate: 1.0000e-05
Epoch 26/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1292s 3s/step - accuracy: 0.6572 - loss: 1.2352 - val_accuracy: 0.5633 - val_loss: 1.5000 - learning_rate: 1.0000e-05
Epoch 27/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1278s

In [26]:
print(history_fine.epoch)


[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


In [28]:
# Continue training the model with fine-tuning
history_fine1 = model.fit(
    {'img_input': X_train_img, 'meta_input': X_train_meta}, 
    y_train,                                                
    validation_data=(
        {'img_input': X_val_img, 'meta_input': X_val_meta},
        y_val
    ),
    epochs=50,
    callbacks=callbacks,
    initial_epoch=len(history_fine.epoch))


Epoch 16/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1276s 2s/step - accuracy: 0.6783 - loss: 1.1827 - val_accuracy: 0.6494 - val_loss: 1.2510 - learning_rate: 2.5000e-06
Epoch 17/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1345s 3s/step - accuracy: 0.6911 - loss: 1.1561 - val_accuracy: 0.6689 - val_loss: 1.1907 - learning_rate: 2.5000e-06
Epoch 18/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 2478s 5s/step - accuracy: 0.6881 - loss: 1.1555 - val_accuracy: 0.6683 - val_loss: 1.2046 - learning_rate: 2.5000e-06
Epoch 19/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1790s 4s/step - accuracy: 0.6960 - loss: 1.1444 - val_accuracy: 0.6803 - val_loss: 1.1688 - learning_rate: 2.5000e-06
Epoch 20/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 2277s 4s/step - accuracy: 0.6904 - loss: 1.1454 - val_accuracy: 0.6828 - val_loss: 1.1628 - learning_rate: 2.5000e-06
Epoch 21/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1871s 4s/step - accuracy: 0.6965 - loss: 1.1491 - val_accuracy: 0.6642 - val_loss: 1.2176 - learning_rate: 2.5000e-06
Epoch 22/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1394s

In [29]:
history_fine2 = model.fit(
    {'img_input': X_train_img, 'meta_input': X_train_meta}, 
    y_train,                                                
    validation_data=(
        {'img_input': X_val_img, 'meta_input': X_val_meta},
        y_val
    ),
    epochs=50,
    callbacks=callbacks,
    initial_epoch=len(history_fine.epoch))

Epoch 16/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1838s 4s/step - accuracy: 0.7008 - loss: 1.1364 - val_accuracy: 0.6842 - val_loss: 1.1386 - learning_rate: 1.0000e-06
Epoch 17/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1383s 3s/step - accuracy: 0.6948 - loss: 1.1345 - val_accuracy: 0.6814 - val_loss: 1.1546 - learning_rate: 1.0000e-06
Epoch 18/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 2821s 6s/step - accuracy: 0.7079 - loss: 1.1169 - val_accuracy: 0.6851 - val_loss: 1.1497 - learning_rate: 1.0000e-06
Epoch 19/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1688s 3s/step - accuracy: 0.7018 - loss: 1.1349 - val_accuracy: 0.6828 - val_loss: 1.1446 - learning_rate: 1.0000e-06
Epoch 20/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1429s 3s/step - accuracy: 0.7008 - loss: 1.1161 - val_accuracy: 0.6767 - val_loss: 1.1751 - learning_rate: 1.0000e-06
Epoch 21/50
509/509 ━━━━━━━━━━━━━━━━━━━━ 1392s 3s/step - accuracy: 0.6956 - loss: 1.1370 - val_accuracy: 0.6775 - val_loss: 1.1799 - learning_rate: 1.0000e-06
